In [1]:
import tensorflow as tf

import os
import time
import cv2
import numpy as np
from glob import glob

from matplotlib import pyplot as plt
from IPython import display

In [2]:
BUFFER_SIZE = 400
BATCH_SIZE = 1
IMG_WIDTH = 256
IMG_HEIGHT = 256

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [15]:
image_files = glob('train_dataset/train_pics/*.*')
image_files_t = glob('train_dataset/test_pics/*.*')
print(len(image_files),len(image_files_t))

157 17


In [5]:
from PIL import Image

def make_square(im, min_size=256, fill_color=(0, 0, 0, 0)):
    x, y = im.size
    size = max(min_size, x, y)
    new_im = Image.new('RGBA', (size, size), fill_color)
    new_im.paste(im, (int((size - x) / 2), int((size - y) / 2)))
    return new_im

In [18]:
# test datasets
n = 0
blocked_layers_t = []
mask_layers_t = []
full_mask_layers_t = []
for file in image_files_t:
    image = Image.open(file)
    # image = cv2.imread(file)
    new_image = make_square(image)
    new_image.save('square_pics/%d.png' %n)
    image = cv2.imread('square_pics/%d.png' %n)
    image = cv2.resize(image, (256, 256))
    m = 0
    while m < 4:
        if m > 0:
            image = cv2.flip(image, m-2)

        height = image.shape[0]
        width = image.shape[1]

        hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        mask1 = cv2.inRange(hsv_image, (0,20,0), (255,255,255)).reshape(256,256,1)
        mask1_3c = np.concatenate((mask1, mask1, mask1), axis=-1)#########here!
        full_mask_layers_t.append(mask1_3c)#####!!!!here

        background = image.copy()
        n_apples = np.random.randint(2, 5)
        n_leaves = np.random.randint(3, 7)
        branchIndex = np.array(np.where(np.equal(mask1, 255)))
        branchIndex = np.transpose([branchIndex[0], branchIndex[1]])
    
        for i in range(n_apples):
            centerpixel = branchIndex[np.random.choice(len(branchIndex))]
            centerpixel = tuple(centerpixel)
            cv2.circle(background, (centerpixel[1], centerpixel[0]), np.random.randint(min(height, width)//30, min(height, width)//15), (0, 0, 255), -1)
        for i in range(n_leaves):
            centerpixel = branchIndex[np.random.choice(len(branchIndex))]
            centerpixel = tuple(centerpixel)
            longaxis = np.random.randint(min(height, width)//20, min(height, width)//10)
            cv2.ellipse(background, (centerpixel[1], centerpixel[0]),  (longaxis, longaxis//2), np.random.choice(360), 0, 360, (0, 255, 0), -1) 
        
        blocked_layers_t.append(background)

        hsv_image_l = cv2.cvtColor(background, cv2.COLOR_BGR2HSV)
        mask1_l = cv2.inRange(hsv_image_l, (0,20,0), (255,250,255))
        mask_apple = cv2.inRange(background, (0,0,254), (0,0,255))
        mask_leaves = cv2.inRange(background, (0,254,0), (0,255,0))

        mask_flatten = mask1_l.flatten()
        maska_flatten = mask_apple.flatten()
        maskl_flatten = mask_leaves.flatten()
        mask_3c = np.zeros(mask_flatten.shape)
        for i in range(len(mask_flatten)):
          if mask_flatten[i] > 0:
            mask_3c[i] = 1
        for i in range(len(maska_flatten)):
          if maska_flatten[i] > 0:
            mask_3c[i] = 2
        for i in range(len(maskl_flatten)):
          if maskl_flatten[i] > 0:
            mask_3c[i] = 2
        mask_3c = mask_3c.reshape(256,256, 1)
        mask_layers_t.append(mask_3c)
        m += 1
        
    n += 1
    
blocked_layers_t = np.array(blocked_layers_t).astype(np.float32) / 127.5 - 1
full_mask_layers_t = np.array(full_mask_layers_t).astype(np.float32) / 127.5 - 1
# mask_layers = np.array(mask_layers)

In [19]:
# train datasets
n = 0
blocked_layers = []
mask_layers = []
full_mask_layers = []
for file in image_files:
    image = Image.open(file)
    # image = cv2.imread(file)
    new_image = make_square(image)
    new_image.save('square_pics/%d.png' %n)
    image = cv2.imread('square_pics/%d.png' %n)
    image = cv2.resize(image, (256, 256))
    m = 0
    while m < 4:
        if m > 0:
            image = cv2.flip(image, m-2)

        height = image.shape[0]
        width = image.shape[1]

        hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        mask1 = cv2.inRange(hsv_image, (0,20,0), (255,255,255)).reshape(256,256,1)
        mask1_3c = np.concatenate((mask1, mask1, mask1), axis=-1)#########here!
        full_mask_layers.append(mask1_3c)#####!!!!here

        background = image.copy()
        n_apples = np.random.randint(2, 5)
        n_leaves = np.random.randint(3, 7)
        branchIndex = np.array(np.where(np.equal(mask1, 255)))
        branchIndex = np.transpose([branchIndex[0], branchIndex[1]])
    
        for i in range(n_apples):
            centerpixel = branchIndex[np.random.choice(len(branchIndex))]
            centerpixel = tuple(centerpixel)
            cv2.circle(background, (centerpixel[1], centerpixel[0]), np.random.randint(min(height, width)//30, min(height, width)//15), (0, 0, 255), -1)
        for i in range(n_leaves):
            centerpixel = branchIndex[np.random.choice(len(branchIndex))]
            centerpixel = tuple(centerpixel)
            longaxis = np.random.randint(min(height, width)//20, min(height, width)//10)
            cv2.ellipse(background, (centerpixel[1], centerpixel[0]),  (longaxis, longaxis//2), np.random.choice(360), 0, 360, (0, 255, 0), -1) 
        
        blocked_layers.append(background)

        hsv_image_l = cv2.cvtColor(background, cv2.COLOR_BGR2HSV)
        mask1_l = cv2.inRange(hsv_image_l, (0,20,0), (255,250,255))
        mask_apple = cv2.inRange(background, (0,0,254), (0,0,255))
        mask_leaves = cv2.inRange(background, (0,254,0), (0,255,0))

        mask_flatten = mask1_l.flatten()
        maska_flatten = mask_apple.flatten()
        maskl_flatten = mask_leaves.flatten()
        mask_3c = np.zeros(mask_flatten.shape)
        for i in range(len(mask_flatten)):
          if mask_flatten[i] > 0:
            mask_3c[i] = 1
        for i in range(len(maska_flatten)):
          if maska_flatten[i] > 0:
            mask_3c[i] = 2
        for i in range(len(maskl_flatten)):
          if maskl_flatten[i] > 0:
            mask_3c[i] = 2
        mask_3c = mask_3c.reshape(256,256, 1)
        mask_layers.append(mask_3c)
        m += 1
        
    n += 1
    
blocked_layers = np.array(blocked_layers).astype(np.float32) / 127.5 - 1
full_mask_layers = np.array(full_mask_layers).astype(np.float32) / 127.5 - 1
mask_layers = np.array(mask_layers)

In [24]:
isinstance(blocked_layers[0], np.float)

False

In [25]:
blocked_layers.shape

(628, 256, 256, 3)

In [26]:
OUTPUT_CHANNELS = 3

In [27]:
def downsample(filters, size, apply_batchnorm=True):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
      tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',
                             kernel_initializer=initializer, use_bias=False))

  if apply_batchnorm:
    result.add(tf.keras.layers.BatchNormalization())

  result.add(tf.keras.layers.LeakyReLU())

  return result

In [28]:
down_model = downsample(3, 4)


In [29]:
def upsample(filters, size, apply_dropout=False):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
    tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                    padding='same',
                                    kernel_initializer=initializer,
                                    use_bias=False))

  result.add(tf.keras.layers.BatchNormalization())

  if apply_dropout:
      result.add(tf.keras.layers.Dropout(0.5))

  result.add(tf.keras.layers.ReLU())

  return result

In [30]:
up_model = upsample(3, 4)

In [31]:
def Generator():
  inputs = tf.keras.layers.Input(shape=[256,256,3])

  down_stack = [
    downsample(64, 4, apply_batchnorm=False), # (bs, 128, 128, 64)
    downsample(128, 4), # (bs, 64, 64, 128)
    downsample(256, 4), # (bs, 32, 32, 256)
    downsample(512, 4), # (bs, 16, 16, 512)
    downsample(512, 4), # (bs, 8, 8, 512)
    downsample(512, 4), # (bs, 4, 4, 512)
    downsample(512, 4), # (bs, 2, 2, 512)
    downsample(512, 4), # (bs, 1, 1, 512)
  ]

  up_stack = [
    upsample(512, 4, apply_dropout=True), # (bs, 2, 2, 1024)
    upsample(512, 4, apply_dropout=True), # (bs, 4, 4, 1024)
    upsample(512, 4, apply_dropout=True), # (bs, 8, 8, 1024)
    upsample(512, 4), # (bs, 16, 16, 1024)
    upsample(256, 4), # (bs, 32, 32, 512)
    upsample(128, 4), # (bs, 64, 64, 256)
    upsample(64, 4), # (bs, 128, 128, 128)
  ]

  initializer = tf.random_normal_initializer(0., 0.02)
  last = tf.keras.layers.Conv2DTranspose(OUTPUT_CHANNELS, 4,
                                         strides=2,
                                         padding='same',
                                         kernel_initializer=initializer,
                                         activation='tanh') # (bs, 256, 256, 3)

  x = inputs

  # Downsampling through the model
  skips = []
  for down in down_stack:
    x = down(x)
    skips.append(x)

  skips = reversed(skips[:-1])

  # Upsampling and establishing the skip connections
  for up, skip in zip(up_stack, skips):
    x = up(x)
    x = tf.keras.layers.Concatenate()([x, skip])

  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)

In [32]:
generator = Generator()

In [33]:
LAMBDA = 100

In [34]:
def generator_loss(disc_generated_output, gen_output, target):
  gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)

  # mean absolute error
  l1_loss = tf.reduce_mean(tf.abs(target - gen_output))

  total_gen_loss = gan_loss + (LAMBDA * l1_loss)

  return total_gen_loss, gan_loss, l1_loss

In [35]:
def Discriminator():
  initializer = tf.random_normal_initializer(0., 0.02)

  inp = tf.keras.layers.Input(shape=[256, 256, 3], name='input_image')
  tar = tf.keras.layers.Input(shape=[256, 256, 3], name='target_image')

  x = tf.keras.layers.concatenate([inp, tar]) # (bs, 256, 256, channels*2)

  down1 = downsample(64, 4, False)(x) # (bs, 128, 128, 64)
  down2 = downsample(128, 4)(down1) # (bs, 64, 64, 128)
  down3 = downsample(256, 4)(down2) # (bs, 32, 32, 256)

  zero_pad1 = tf.keras.layers.ZeroPadding2D()(down3) # (bs, 34, 34, 256)
  conv = tf.keras.layers.Conv2D(512, 4, strides=1,
                                kernel_initializer=initializer,
                                use_bias=False)(zero_pad1) # (bs, 31, 31, 512)

  batchnorm1 = tf.keras.layers.BatchNormalization()(conv)

  leaky_relu = tf.keras.layers.LeakyReLU()(batchnorm1)

  zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu) # (bs, 33, 33, 512)

  last = tf.keras.layers.Conv2D(1, 4, strides=1,
                                kernel_initializer=initializer)(zero_pad2) # (bs, 30, 30, 1)

  return tf.keras.Model(inputs=[inp, tar], outputs=last)

In [36]:
discriminator = Discriminator()

In [37]:
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [38]:
def discriminator_loss(disc_real_output, disc_generated_output):
  real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)

  generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)

  total_disc_loss = real_loss + generated_loss

  return total_disc_loss

In [39]:
generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

In [40]:
checkpoint_dir = 'training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [53]:
#     idx = np.random.randint(0, len(blocked_layers))
#     o_img = blocked_layers[idx].reshape(1,256,256,3)
#     p_mask = (np.array(generator(o_img)).reshape(256,256,3)+1)*127.5
#     f_mask = (full_mask_layers[idx]+1)*127.5
#     o_img = (o_img.reshape(256,256,3)+1)*127.5

#     vis = np.concatenate((o_img, p_mask, f_mask), axis=1)
#     cv2.imwrite('pix2pix_images/1.png' , vis)

True

In [46]:
if not os.path.exists('pix2pix_images'):
  os.makedirs('pix2pix_images')

def sample_images(epoch):
    idx = np.random.randint(0, len(blocked_layers_t))
    o_img = blocked_layers_t[idx].reshape(1,256,256,3)
    p_mask = (np.array(generator(o_img)).reshape(256,256,3)+1)*127.5
    f_mask = (full_mask_layers_t[idx]+1)*127.5
    o_img = (o_img.reshape(256,256,3)+1)*127.5

    vis = np.concatenate((o_img, p_mask, f_mask), axis=1)
    cv2.imwrite('pix2pix_images/%d.png' % epoch, vis)

In [42]:
EPOCHS = 150

In [48]:
@tf.function
def train_step(input_image, target, epoch):
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    gen_output = generator(input_image, training=True)

    disc_real_output = discriminator([input_image, target], training=True)
    disc_generated_output = discriminator([input_image, gen_output], training=True)

    gen_total_loss, gen_gan_loss, gen_l1_loss = generator_loss(disc_generated_output, gen_output, target)
    disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

  generator_gradients = gen_tape.gradient(gen_total_loss,
                                          generator.trainable_variables)
  discriminator_gradients = disc_tape.gradient(disc_loss,
                                               discriminator.trainable_variables)

  generator_optimizer.apply_gradients(zip(generator_gradients,
                                          generator.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(discriminator_gradients,
                                              discriminator.trainable_variables))

#   with summary_writer.as_default():
#     tf.summary.scalar('gen_total_loss', gen_total_loss, step=epoch)
#     tf.summary.scalar('gen_gan_loss', gen_gan_loss, step=epoch)
#     tf.summary.scalar('gen_l1_loss', gen_l1_loss, step=epoch)
#     tf.summary.scalar('disc_loss', disc_loss, step=epoch)

In [49]:
def fit(blocked_layers, full_mask_layers, epochs, blocked_layers_t, full_mask_layers_t):
  for epoch in range(epochs):
    start = time.time()

#     display.clear_output(wait=True)

#     for example_input, example_target in test_ds.take(1):
#       generate_images(generator, example_input, example_target)
    sample_images(epoch)
    print("Epoch: ", epoch)
    
    # Train
    for n in range(len(blocked_layers)):
        print('.', end='')
        if (n+1) % 100 == 0:
            print()
        train_step(blocked_layers[n].reshape(1,256,256,3), full_mask_layers[n].reshape(1,256,256,3), epoch)
    print()
    # Train
#     for n, (input_image, target) in train_ds.enumerate():
#       print('.', end='')
#       if (n+1) % 100 == 0:
#         print()
#       train_step(input_image, target, epoch)
#     print()

    # saving (checkpoint) the model every 20 epochs
    if (epoch + 1) % 20 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                        time.time()-start))
  checkpoint.save(file_prefix = checkpoint_prefix)

In [50]:
fit(blocked_layers, full_mask_layers, EPOCHS, blocked_layers_t, full_mask_layers_t)

Epoch:  0
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
............................
Time taken for epoch 1 is 91.89574313163757 sec

Epoch:  1
....................................................................................................
....................................................................................................
..............................................................................................

....................................................................................................
....................................................................................................
....................................................................................................
............................
Time taken for epoch 24 is 90.12797331809998 sec

Epoch:  24
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
.

Epoch:  47
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
............................
Time taken for epoch 48 is 89.25284695625305 sec

Epoch:  48
....................................................................................................
....................................................................................................
...........................................................................................

....................................................................................................
....................................................................................................
....................................................................................................
............................
Time taken for epoch 71 is 89.06791281700134 sec

Epoch:  71
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
.

Epoch:  94
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
............................
Time taken for epoch 95 is 88.93775129318237 sec

Epoch:  95
....................................................................................................
....................................................................................................
...........................................................................................

....................................................................................................
....................................................................................................
....................................................................................................
............................
Time taken for epoch 118 is 91.97383141517639 sec

Epoch:  118
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................

............................
Time taken for epoch 141 is 99.95843505859375 sec

Epoch:  141
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
............................
Time taken for epoch 142 is 93.0608777999878 sec

Epoch:  142
....................................................................................................
....................................................................................................
.........